In [2]:
# ================= Regular/Postseason ESPN + Live CSV Writer =================
!pip -q install pandas requests

import time, tempfile, shutil
from pathlib import Path
import numpy as np
import pandas as pd
import requests

# -------- CONFIG --------
SEASON      = 2025          # e.g., 2025
SEASONTYPE  = 2             # 2 = Regular, 3 = Postseason
WEEK        = 1             # NFL week number
POLL_SEC    = 60            # how often to refresh the CSV (seconds)

DATA_DIR = Path("data"); DATA_DIR.mkdir(parents=True, exist_ok=True)
OUT_CSV  = DATA_DIR / "player_stats.csv"

# ESPN endpoints
SCORE_URL = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard"
BOX_URL   = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/boxscore"
SUM_URL   = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/summary"

# Stable schema for Tableau (includes season/week to avoid errors)
BASE_COLS = [
    "event_id","team","player","position","stat_type",
    "passAtt","passCmp","passYds","passTD","int",
    "rushAtt","rushYds","rushTD",
    "rec","recYds","recTD","yac",
    "season","week",
]

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={c: str(c).strip().lower() for c in df.columns})
    for c in BASE_COLS:
        if c not in df.columns:
            df[c] = np.nan
    return df[BASE_COLS]

def atomic_write_csv(df: pd.DataFrame, path: Path):
    with tempfile.NamedTemporaryFile("w", delete=False, suffix=".csv", newline="") as tmp:
        df.to_csv(tmp.name, index=False)
        tmp_path = Path(tmp.name)
    shutil.move(str(tmp_path), str(path))

def _to_num(x):
    try: return float(x)
    except Exception: return x

def _stats_to_dict(stats):
    if isinstance(stats, dict):
        return {k:_to_num(v) for k,v in stats.items()}
    out = {}
    if isinstance(stats, list):
        for it in stats:
            if isinstance(it, dict):
                name = it.get("name") or it.get("shortDisplayName") or it.get("displayName")
                if name:
                    out[name] = _to_num(it.get("value", it.get("displayValue")))
    return out

def _athlete_fields(ath):
    if isinstance(ath, dict):
        return ath.get("displayName"), (ath.get("position") or {}).get("abbreviation")
    return None, None

def _row(event_id, team_abbr, name, pos, cat, sd, season, week):
    cat = (cat or "").lower()
    return {
        "event_id": event_id, "team": team_abbr, "player": name, "position": pos, "stat_type": cat,
        # Passing
        "passAtt": sd.get("attempts") if cat=="passing" else np.nan,
        "passCmp": sd.get("completions") if cat=="passing" else np.nan,
        "passYds": sd.get("yards") if cat=="passing" else np.nan,
        "passTD":  sd.get("touchdowns") if cat=="passing" else np.nan,
        "int":     sd.get("interceptions") if cat=="passing" else np.nan,
        # Rushing
        "rushAtt": sd.get("attempts") if cat=="rushing" else np.nan,
        "rushYds": sd.get("yards") if cat=="rushing" else np.nan,
        "rushTD":  sd.get("touchdowns") if cat=="rushing" else np.nan,
        # Receiving
        "rec":     sd.get("receptions") if cat=="receiving" else np.nan,
        "recYds":  sd.get("yards") if cat=="receiving" else np.nan,
        "recTD":   sd.get("touchdowns") if cat=="receiving" else np.nan,
        "yac":     sd.get("yardsAfterCatch", sd.get("yac")) if cat=="receiving" else np.nan,
        "season": season, "week": week,
    }

def get_events(season:int, seasontype:int, week:int):
    r = requests.get(SCORE_URL, params={"seasontype":seasontype, "week":week, "limit":2000, "dates":season}, timeout=25)
    r.raise_for_status()
    js = r.json()
    return [str(e.get("id")) for e in (js.get("events") or []) if isinstance(e, dict) and e.get("id")]

def _extract_from_box(js, event_id, season, week):
    rows = []
    box = js.get("boxscore") or js
    for block in ("players","teams"):            # ESPN flips shapes
        arr = (box.get(block) or [])
        if not isinstance(arr, list): 
            continue
        for entry in arr:
            if not isinstance(entry, dict): 
                continue
            team_abbr = ((entry.get("team") or {}).get("abbreviation"))
            for cat in (entry.get("statistics") or []):
                if not isinstance(cat, dict): 
                    continue
                cat_name = cat.get("name")
                for ath in (cat.get("athletes") or []):
                    if not isinstance(ath, dict): 
                        continue
                    name, pos = _athlete_fields(ath.get("athlete"))
                    if not name: 
                        continue
                    sd = _stats_to_dict(ath.get("stats"))
                    rows.append(_row(event_id, team_abbr, name, pos, cat_name, sd, season, week))
    return pd.DataFrame(rows)

def fetch_week(season:int, seasontype:int, week:int) -> pd.DataFrame:
    ids = get_events(season, seasontype, week)
    frames = []
    for eid in ids:
        df = pd.DataFrame()
        try:
            r = requests.get(BOX_URL, params={"event":eid}, timeout=25)
            if r.status_code == 200:
                df = _extract_from_box(r.json(), eid, season, week)
        except Exception:
            pass
        if df.empty:
            try:
                df = _extract_from_box(requests.get(SUM_URL, params={"event":eid}, timeout=25).json(),
                                       eid, season, week)
            except Exception:
                df = pd.DataFrame()
        if not df.empty:
            frames.append(df)
    if not frames:
        return pd.DataFrame(columns=BASE_COLS)

    out = pd.concat(frames, ignore_index=True)

    # Keep only QB / RB / WR / TE
    pos_map = {
        "QB":"QB",
        "RB":"RB","HB":"RB","FB":"RB",
        "WR":"WR","PR":"WR","KR":"WR","FL":"WR","SL":"WR",
        "TE":"TE",
    }
    out["position"] = out["position"].map(pos_map)
    out = out[out["position"].isin(["QB","RB","WR","TE"])].copy()

    # Stable schema + fill season/week
    out["season"] = season
    out["week"]   = week
    out = ensure_schema(out)
    return out

def build_once_and_write():
    if SEASONTYPE not in (2, 3):
        raise ValueError("SEASONTYPE must be 2 (Regular) or 3 (Postseason).")
    df = fetch_week(SEASON, SEASONTYPE, WEEK)
    atomic_write_csv(df, OUT_CSV)
    print(f"Wrote {OUT_CSV.resolve()} | rows={len(df)}")
    return df

def live_loop():
    print(f"Live writing to: {OUT_CSV.resolve()}  (Ctrl+C / Kernel→Interrupt to stop)")
    try:
        while True:
            try:
                df = build_once_and_write()
                print(f"[{time.strftime('%H:%M:%S')}] rows={len(df)}")
            except Exception as e:
                print(f"[{time.strftime('%H:%M:%S')}] ERROR: {e}")
            time.sleep(POLL_SEC)
    except KeyboardInterrupt:
        print("Stopped live updates.")


In [3]:
df = build_once_and_write()
df.head(12)

Wrote C:\Users\noahh\NFL Live Tracker\data\player_stats.csv | rows=0


,event_id,team,player,position,stat_type,passAtt,passCmp,passYds,passTD,int,rushAtt,rushYds,rushTD,rec,recYds,recTD,yac,season,week


In [ ]:
live_loop()

Live writing to: C:\Users\noahh\NFL Live Tracker\data\player_stats.csv  (Ctrl+C / Kernel→Interrupt to stop)
Wrote C:\Users\noahh\NFL Live Tracker\data\player_stats.csv | rows=0
[11:17:56] rows=0
Wrote C:\Users\noahh\NFL Live Tracker\data\player_stats.csv | rows=0
[11:19:01] rows=0
Wrote C:\Users\noahh\NFL Live Tracker\data\player_stats.csv | rows=0
[11:20:06] rows=0


In [ ]:
# 1) Pull once
df = build_once_and_write()
print(len(df), "rows")
print(df.columns.tolist())
df.head(3)